In [1]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os
import imageio
import tqdm
import time

MyoSuite:> Registering Myo Envs


In [2]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


<h2> Create an A matrix with T x N (time x mean firing rate)

<h3> Get Policy mjrl

In [3]:
policy = "models/LSTM_Trained/iterations/best_policy.pickle" #Change the folder to the policy location

import pickle
# load policy
pi = pickle.load(open(policy, 'rb'))


<h3> Get Policy sb3

In [8]:
from stable_baselines3 import PPO
import os
from sb3_contrib import RecurrentPPO

env = gym.make('CenterReachOut-v0')

model = RecurrentPPO.load("test", env=env)

<h2> Setup data collection for mjrl policy </h2>

In [17]:
# Use sleep if you want to examine certain steps during the episodes
from time import sleep
import torch

env = gym.make('CenterReachOut-v0')
frames = [] 


NeuralData = {}
MuscleActData = {}
FiberJointData = {}
FiberData = {}
JointData = {}

conditions = {}

for ep_num in range(108):  #Number of episodes
  NeuralData[ep_num] = []
  MuscleActData[ep_num] = []
  FiberJointData[ep_num] = []
  FiberData[ep_num] = []
  JointData[ep_num] = []
  
  timestep = 0
  env.reset()
  print(ep_num)
  ep_rewards = []
  done = False
  obs = env.reset()
  

  for ac_num in range(16): # Number of steps per episode / Each time step is 0.05 seconds
      timestep += 0.05
      frame = env.sim.renderer.render_offscreen(
                        width=400,
                        height=400,
                        camera_id=1)
      frames.append(frame)
      # We're appending data of each timestep to an epsiode number key
      o = env.get_obs()
      action, _ = pi.get_action(o)
      # For the mjrl policy 
      # For OG_Model 
      #NeuralData[ep_num].append(np.concatenate((pi.show_activations()["layer_0"][0],pi.show_activations()["layer_1"][0] )))

      # For LSTM (Both layers)
      #NeuralData[ep_num].append(np.concatenate((pi.show_activations()[0].detach().numpy()[0][0],pi.show_activations()[1].detach().numpy()[0])))
      # For LSTM (Only the recurrent layer)
      NeuralData[ep_num].append(pi.show_activations()[0].detach().numpy()[0][0])

      #NeuralData[ep_num].append(torch.cat((model.policy.activations[0], model.policy.activations[1]), dim=0).flatten().cpu().numpy())
      MuscleActData[ep_num].append(env.obs_dict['act'])
      FiberJointData[ep_num].append(np.concatenate((env.obs_dict['hand_qpos'],env.obs_dict['hand_qvel'], env.sim.data.actuator_length[:].copy(), env.sim.data.actuator_velocity[:].copy())))
      FiberData[ep_num].append(np.concatenate((env.obs_dict['hand_qpos'], env.obs_dict['hand_qvel'])))
      JointData[ep_num].append(np.concatenate((env.sim.data.actuator_length[:].copy(), env.sim.data.actuator_velocity[:].copy())))

      obs, reward, done, info = env.step(action)
            

env.close()


os.makedirs('videos', exist_ok=True)
video_path = 'videos/test_policy.mp4'
# make a local copy
imageio.mimsave(video_path, frames, fps=30)
show_video('videos/test_policy.mp4')

[-0.3680450158763584, -0.07925561667782058, 1.07]
[-0.3680450158763584, -0.07925561667782058, 1.07]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [26]:
print(f"Neural Data: {NeuralData[0][1]} \n length: {len(NeuralData[0][1])}") # 512 Neurons
print(f"Muscle Actuator Data: {MuscleActData[0][1]}\n length: {len(MuscleActData[0][1])}") # 63 Muscles
print(f"Fiber/Joint Angle/Joint Velocity Data: {FiberJointData[0][1]} \n length: {len(FiberJointData[0][1])}") # 63 muscles + 44 joints
print(f"Joint Angle/Joint Velocity Data: {FiberData[0][1]} \n length: {len(FiberData[0][1])}") 
print(f"Fiber intensity/Velocity Data: {JointData[0][1]} \n length: {len(JointData[0][1])}") 

Neural Data: [0.0699 0.2263 0.2639 0.3672 0.135  0.0479 0.1216 0.191  0.1521 0.1543
 0.0735 0.2537 0.0823 0.1469 0.1185 0.1276 0.5803 0.0714 0.12   0.2026
 0.3774 0.282  0.1012 0.177  0.1053 0.2013 0.085  0.1249 0.2088 0.1908
 0.6802 0.4429 0.23   0.0564 0.5215 0.1598 0.7503 0.453  0.4205 0.2658
 0.0839 0.0982 0.1315 0.2494 0.0608 0.2237 0.2344 0.2752 0.2765 0.4951
 0.1193 0.844  0.1674 0.2272 0.6186 0.0842 0.1956 0.9057 0.092  0.6061
 0.1539 0.5453 0.9172 0.4656 0.0618 0.395  0.4264 0.1443 0.0951 0.8505
 0.238  0.1088 0.4162 0.4441 0.1811 0.6551 0.198  0.1095 0.0645 0.7039
 0.3572 0.703  0.1712 0.6509 0.1852 0.0767 0.0927 0.469  0.2601 0.8526
 0.0877 0.0737 0.2538 0.3513 0.1138 0.4622 0.1655 0.1293 0.1407 0.1892
 0.1801 0.8052 0.0818 0.1519 0.4684 0.1394 0.1194 0.5573 0.2554 0.1799
 0.3411 0.1922 0.186  0.325  0.2227 0.3433 0.2472 0.0726 0.0699 0.1667
 0.1668 0.0876 0.0594 0.1144 0.2453 0.071  0.1185 0.2639 0.1378 0.1467
 0.1844 0.1662 0.3511 0.035  0.1095 0.2165 0.1409 0.1284 0.2471 

In [27]:
import numpy as np
import scipy.io

time_steps = 16

# Create the `times` array and ensure it's double precision
times =  np.arange(0, time_steps*0.05, 0.05).reshape(-1, 1)
    
data = [NeuralData, MuscleActData, FiberJointData, FiberData, JointData]
matrices = []

for x in range(len(data)): # 3 Matrices
    data_struct = np.empty((ep_num,), dtype=[('A', 'O'), ('times', 'O')])

    for n in range(ep_num):
        data_struct['A'][n] = data[x][n] 
        data_struct['times'][n] = times  

    
    matrices.append(data_struct)
    
# Save the structured data to a .mat file with 'Data' as the matrix name
scipy.io.savemat('OG_Model_neural_data.mat', {'Data': matrices[0]})
scipy.io.savemat('OG_Model_muscle_data.mat', {'Data': matrices[1]})
scipy.io.savemat('OG_Model_fiber_joint_data.mat', {'Data': matrices[2]})
scipy.io.savemat('OG_Model_fiber_data.mat', {'Data': matrices[3]})
scipy.io.savemat('OG_Model_joint_data.mat', {'Data': matrices[4]})



<p> We'll basically have 3 matrices A (number of rows = number of steps): <br>
    - Neurons: 512 columns for each neuron <br>
    - Muscle Activity: 63 columns for each muscle actuator intensity <br>
    - Fiber Length/Joint Angle/Velocity: 151 columns for combination of 63 fiber length, 44 joint angle and 44 joint velocities <br>

# Down Sampling
<p> Since the amount of neurons, muscles and fiber/joints are not equal, we want to make that we account for that size difference. 

In [14]:
print(len(data[2][0][0]))
# neural data: len(data[0][ep_1, ep_2 ... ep108][act_1, act_2...act16]) = 512
# muscle data: len(data[1][ep_1, ep_2 ... ep108][act_1, act_2...act16]) = 63
# fiber data: len(data[2][ep_1, ep_2 ... ep108][act_1, act_2...act16]) = 214

test_neuraldata = list(data[0].values())
test_muscledata = list(data[1].values())
test_fiberdata = list(data[2].values())

214


In [15]:
def downsample(dataset):
    downsampled_dataset = []
    for episode in dataset:
        downsampled_episode = []
        for activity in episode:
            if len(activity) > 63:
                step = len(activity) // 63
                downsampled_activity = activity[::step][:63]  
            else:
                downsampled_activity = activity 
            downsampled_episode.append(downsampled_activity)
        downsampled_dataset.append(downsampled_episode)
    return downsampled_dataset

In [16]:
downsampled_Neural_data = {}

downsampled_Neural_data = downsample(test_neuraldata)

downsampled_Neural_data = {index: value for index, value in enumerate(downsampled_Neural_data)}

In [17]:
downsampled_FiberJoint_data = {}

downsampled_FiberJoint_data = downsample(test_fiberdata)
    
downsampled_FiberJoint_data = {index: value for index, value in enumerate(downsampled_FiberJoint_data )}

print(len(downsampled_Neural_data[0][0]))

63


Create Matrix

In [19]:
import numpy as np
import scipy.io

time_steps = 16

# Create the `times` array and ensure it's double precision
times =  np.arange(0, time_steps*0.05, 0.05).reshape(-1, 1)
    
data = [downsampled_Neural_data, MuscleActData, downsampled_FiberJoint_data]
matrices = []

for x in range(3): # 3 Matrices
    data_struct = np.empty((ep_num,), dtype=[('A', 'O'), ('times', 'O')])

    for n in range(ep_num):
        data_struct['A'][n] = data[x][n] 
        data_struct['times'][n] = times  

    
    matrices.append(data_struct)
    
# Save the structured data to a .mat file with 'Data' as the matrix name
scipy.io.savemat('Neural_Data_downsampled_training.mat', {'Data': matrices[0]})
scipy.io.savemat('MuscleAct_Data_downsampled_training.mat', {'Data': matrices[1]})
scipy.io.savemat('FiberJoint_Data_downsampled_training.mat', {'Data': matrices[2]})
scipy.io.savemat('Fiber_Data_downsampled_training.mat', {'Data': matrices[3]})
scipy.io.savemat('Joint_Data_downsampled_training.mat', {'Data': matrices[4]})


IndexError: list index out of range